In [1]:
#!pip install rake_nltk

In [125]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('comedy-movie-database.csv')

df = df[['Title','Genre','Director','Actors','Plot']]
df=df[:51]

df=df.set_index("Title")
print(df.head())

                                                  Genre          Director  \
Title                                                                       
Jojo Rabbit              Comedy, Drama, War                 Taika Waititi   
Guns Akimbo        Action, Comedy, Thriller              Jason Lei Howden   
Thor: Ragnarok    Action, Adventure, Comedy                 Taika Waititi   
Charlie's Angels  Action, Adventure, Comedy               Elizabeth Banks   
The Truman Show       Comedy, Drama, Sci-Fi                    Peter Weir   

                                                             Actors  \
Title                                                                 
Jojo Rabbit       ['Roman Griffin Davis', 'Thomasin McKenzie', '...   
Guns Akimbo       ['Samara Weaving', 'Daniel Radcliffe', 'Rhys D...   
Thor: Ragnarok    ['Chris Hemsworth', 'Tom Hiddleston', 'Cate Bl...   
Charlie's Angels  ['Kristen Stewart', 'Naomi Scott', 'Ella Balin...   
The Truman Show   ['Jim Carrey', '

In [126]:
# initializing the new column
df['Key_words'] = ""
#print(df["Actors"])

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

In [127]:
df['Actors'] = [word.lstrip('[') for word in df['Actors']]
df['Actors'] = [word.rstrip(']') for word in df['Actors']]
df["Actors"] = df["Actors"].str.replace(" ","")
df["Actors"] = df["Actors"].str.replace("\'","")
df["Actors"] = df["Actors"].str.replace(","," ")
df.Actors


Title
Jojo Rabbit                          RomanGriffinDavis ThomasinMcKenzie ScarlettJoh...
Guns Akimbo                          SamaraWeaving DanielRadcliffe RhysDarby NedDen...
Thor: Ragnarok                       ChrisHemsworth TomHiddleston CateBlanchett Mar...
Charlie's Angels                     KristenStewart NaomiScott EllaBalinska Elizabe...
The Truman Show                            JimCarrey EdHarris LauraLinney NoahEmmerich
Airplane!                            DavidZucker JerryZucker RobertHays JulieHagert...
The Wrong Missy                      DavidSpade LaurenLapkus NickSwardson GeoffPierson
Bad Boys for Life                    BilallFallah WillSmith MartinLawrence VanessaH...
Ready or Not                         TylerGillett SamaraWeaving AdamBrody "MarkOBri...
Hustlers                             ConstanceWu JenniferLopez JuliaStiles MetteTowley
Spaceballs                                 MelBrooks JohnCandy RickMoranis BillPullman
Clueless                             

In [128]:

df["Genre"] = df["Genre"].str.lower()
df['Genre'] = [word.rstrip() for word in df['Genre']]
df["Director"] = df["Director"].str.lower()
df["Actors"] = df["Actors"].str.lower()

#df["Genre"] = df["Genre"].str.replace(" ","")
df["Director"] = df["Director"].str.replace(" ","")
#df["Actors"] = df["Actors"].str.replace(" ","")
df["Genre"] = df["Genre"].str.replace(","," ")
#df["Director"] = df["Director"].str.replace(","," ")
#df["Actors"] = df["Actors"].str.replace(","," ")
print(df["Actors"])
df["bag_of_words"]=df["Genre"]+" "+df["Director"]+" "+df["Actors"]+" "+str(df["Key_words"])
df["bag_of_words"] = df["bag_of_words"].str.replace(","," ")
df.head()

Title
Jojo Rabbit                          romangriffindavis thomasinmckenzie scarlettjoh...
Guns Akimbo                          samaraweaving danielradcliffe rhysdarby nedden...
Thor: Ragnarok                       chrishemsworth tomhiddleston cateblanchett mar...
Charlie's Angels                     kristenstewart naomiscott ellabalinska elizabe...
The Truman Show                            jimcarrey edharris lauralinney noahemmerich
Airplane!                            davidzucker jerryzucker roberthays juliehagert...
The Wrong Missy                      davidspade laurenlapkus nickswardson geoffpierson
Bad Boys for Life                    bilallfallah willsmith martinlawrence vanessah...
Ready or Not                         tylergillett samaraweaving adambrody "markobri...
Hustlers                             constancewu jenniferlopez juliastiles mettetowley
Spaceballs                                 melbrooks johncandy rickmoranis billpullman
Clueless                             

,Genre,Director,Actors,Key_words,bag_of_words
Title,,,,,
Jojo Rabbit,comedy drama war,taikawaititi,romangriffindavis thomasinmckenzie scarlettjoh...,"[hiding, hitler, home, mother, army, finds, yo...",comedy drama war taikawaititi romangriffinda...
Guns Akimbo,action comedy thriller,jasonleihowden,samaraweaving danielradcliffe rhysdarby nedden...,"[newly, girlfriend, kidnappers, ex, guy, relie...",action comedy thriller jasonleihowden samara...
Thor: Ragnarok,action adventure comedy,taikawaititi,chrishemsworth tomhiddleston cateblanchett mar...,"[stop, ragnarök, world, destruction, powerful,...",action adventure comedy taikawaititi chrishe...
Charlie's Angels,action adventure comedy,elizabethbanks,kristenstewart naomiscott ellabalinska elizabe...,"[called, protect, us, action, dangerous, techn...",action adventure comedy elizabethbanks krist...
The Truman Show,comedy drama sci-fi,peterweir,jimcarrey edharris lauralinney noahemmerich,"[whole, life, actually, reality, tv, show, ins...",comedy drama sci-fi peterweir jimcarrey edha...


In [129]:
print(df["bag_of_words"])


Title
Jojo Rabbit                          comedy  drama  war taikawaititi romangriffinda...
Guns Akimbo                          action  comedy  thriller jasonleihowden samara...
Thor: Ragnarok                       action  adventure  comedy taikawaititi chrishe...
Charlie's Angels                     action  adventure  comedy elizabethbanks krist...
The Truman Show                      comedy  drama  sci-fi peterweir jimcarrey edha...
Airplane!                            comedy jimabrahams davidzucker jerryzucker rob...
The Wrong Missy                      comedy  romance tylerspindel davidspade lauren...
Bad Boys for Life                    action  comedy  crime adilelarbi bilallfallah ...
Ready or Not                         comedy  horror  mystery mattbettinelli-olpin t...
Hustlers                             comedy  crime  drama lorenescafaria constancew...
Spaceballs                           adventure  comedy  sci-fi melbrooks melbrooks ...
Clueless                             

In [130]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [131]:
print(cosine_sim)


[[1.         0.99044276 0.99285498 ... 0.99161748 0.99041987 0.99101418]
 [0.99044276 1.         0.99285996 ... 0.99162963 0.99162963 0.99222467]
 [0.99285498 0.99285996 1.         ... 0.99165808 0.99165808 0.99225313]
 ...
 [0.99161748 0.99162963 0.99165808 ... 1.         0.99160671 0.99220174]
 [0.99041987 0.99162963 0.99165808 ... 0.99160671 1.         0.9934015 ]
 [0.99101418 0.99222467 0.99225313 ... 0.99220174 0.9934015  1.        ]]


In [132]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)
#print(indices)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]
    #print(idx)

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    #print(score_series)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:51].index)
    #print(top_10_indexes)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(indices)[i])
        #print(indices[i])
        #print(recommended_movies)
        #print(df.index)
    return recommended_movies

In [121]:
recommendations("Pearl Harbor")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [122]:
recommendations("Atonement")

['Blade Runner 2049',
 'Se7en',
 'Prisoners',
 'Double Jeopardy',
 'The Night Clerk',
 'Watchmen',
 'Eyes Wide Shut',
 'Gone Girl',
 'Apocalypse Now',
 'Inheritance',
 'Ad Astra',
 'Hereditary',
 'The Firm',
 'The Hateful Eight',
 'The Vast of Night',
 'The Beach House',
 'The Prestige',
 'Ex Machina',
 'Shutter Island',
 'Cloud Atlas',
 'Murder on the Orient Express',
 'Gretel & Hansel',
 'Sleepy Hollow',
 'Hot Fuzz',
 'The Guest',
 'Red Riding Hood',
 'The Usual Suspects',
 'Psycho',
 'Scary Stories to Tell in the Dark',
 'Get Out',
 'Vivarium',
 'I See You',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Us',
 'Memento',
 'A Simple Favor',
 'The Lighthouse',
 'Freaks',
 'The Thing',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Order of the Phoenix',
 'Sen to Chihiro no kamikakushi',
 'Winchester',
 'Ready or Not',
 'Fantasy Island',
 'Coco',
 'Game Night',
 'Zootopia']

In [123]:
recommendations("Gone Girl")

['Se7en',
 'Eyes Wide Shut',
 'The Firm',
 'Inheritance',
 'Shutter Island',
 'The Usual Suspects',
 'Blade Runner 2049',
 'Prisoners',
 'Double Jeopardy',
 'The Night Clerk',
 'Watchmen',
 'Apocalypse Now',
 'Get Out',
 'Us',
 'Ad Astra',
 'Atonement',
 'Memento',
 'Psycho',
 'Hereditary',
 'The Hateful Eight',
 'Ex Machina',
 'The Beach House',
 'The Prestige',
 'The Vast of Night',
 'Cloud Atlas',
 'Murder on the Orient Express',
 'Gretel & Hansel',
 'Red Riding Hood',
 'Sleepy Hollow',
 'The Guest',
 'Hot Fuzz',
 'Vivarium',
 'A Simple Favor',
 'Scary Stories to Tell in the Dark',
 'I See You',
 'Harry Potter and the Deathly Hallows: Part 2',
 'The Lighthouse',
 'Freaks',
 'The Thing',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Order of the Phoenix',
 'Sen to Chihiro no kamikakushi',
 'Winchester',
 'Ready or Not',
 'Fantasy Island',
 'Coco',
 'Game Night',
 'Zootopia']